!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from movement import *
from numpy.random import default_rng
import pickle

import binascii


import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation
# %matplotlib notebook

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 2
nr_of_genes = 8
nr_individuals = 600

nr_of_generations = 100

## world size
world_size = 100
world_size_x = world_size
world_size_y = world_size

## generations

In [ ]:
time_list = []
result = {}
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        result, a,b,c,d = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
#         print(result[0])
    else:
        
        result = asexual_reproduction_and_mutation(world_size, result, nr_individuals)
        if result:
            result = next_generation(result,nr_of_input, nr_of_actions, nr_of_inner, world_size, nr_individuals)
            result, a,b,c,d  = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)
            time_list.append([a,b,c,d])
    result.update(result)
    
    dic_color = {}
    for indiv in result:
        rgb_colors = list(map(hex_to_rgb, result[indiv]['genome']))
        color = tuple(pd.DataFrame(rgb_colors).median()) 
        dic_color[indiv] = color
        
    coords = generate_dictionary_of_coords(result, world_size*2, dic_color)
    
    def update(i):
        ax.clear()
        ax.set_facecolor(plt.cm.Blues(.2))

        ax.set_xlim([0,world_size_x])
        ax.set_ylim([0,world_size_y])
        ax.set_title('moving')
        ax.scatter(x=coords[i]['x'],y=coords[i]['y'], c=coords[i]['color'], s=20, marker='o')
        [spine.set_visible(False) for spine in ax.spines.values()]


    fig, ax = plt.subplots(figsize=(6,6))
    myAnimation = FuncAnimation(
        fig = fig,
        func = update,
        frames = len(coords),
        interval = 10, repeat=False
    )
    myAnimation.save(f'./output/generation-{gen_nr}.gif', writer='imagemagick' , fps=10)
    # create a binary pickle file 
    f = open(f'./output/generation-{gen_nr}.pkl',"wb")
    pickle.dump(result, f)
    f.close()

In [3]:
for gen_nr in range(nr_of_generations):
    print(gen_nr)
    if gen_nr == 0:
        result = initial_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner, world_size)
        result, a,b,c,d = steps_in_generation(world_size*1.2, result, world_size_x, world_size_y)

0


  0%|                                                                                        | 0/120.0 [00:00<?, ?it/s]

TypeError: calculate_position() takes 5 positional arguments but 6 were given

In [23]:
result[1]

{'out': {'out1': 0.448, 'out4': 0.881, 'out0': 0.001},
 'brain': {'mid1out1': ['mid1', 'out1', -0.10499008087898673],
  'in0mid0': ['in0', 'mid0', 3.3734777964291163],
  'mid0out4': ['mid0', 'out4', 3.335144208759347],
  'mid1out0': ['mid1', 'out0', -3.259453685334961],
  'mid1mid1': ['mid1', 'mid1', -3.0694948878376316],
  'in1mid0': ['in1', 'mid0', -0.9453990538684572],
  'in1mid1': ['in1', 'mid1', 0.5249504043949336]},
 'in': ['in0', 'in1'],
 'genome': ['b590035c',
  '5e2eebf1',
  '9ebbeab7',
  '8bb7684b',
  '81136237',
  'dab8991a',
  '7b241e40',
  '392990cc'],
 'position': [[5, 61]]}

In [65]:
position_list = np.array(list(move(out, result[1]['out'][out]) for out in result[1]['out']))
position_list

array([[ 0, -1],
       [ 1,  1],
       [ 0,  1]])

In [66]:
np.array(result[1]['position'][-1])

array([ 5, 61])

In [53]:

for factor in position_list:
    result[1]['position']

[1]
[1]
[1]
[0]
[1]
[0]
[0]
[1]
[1]
[1]


In [19]:
def calculate_position(result, indiv, x, y, world_size_x, world_size_y):

    position_list = (move(out, result[indiv]['out'][out]) for out in result[indiv]['out'])

    if position_list:
        position_list = make_smaller_(list(map(sum, zip(*position_list))))
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))

        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)

        result[indiv]['position'].append(position_list)
        
def move(key, weight):
    factor_1 = np.random.choice(2, 1, p=[weight, 1-weight])
    if 'out0' in key:
        return [0, int(factor_1)]
    elif 'out1' in key:
        return [0, int(-factor_1)]
    elif 'out2' in key:
        return [int(+factor_1), 0]
    elif 'out3' in key:
        return [int(-factor_1), 0]
    elif 'out4' in key:
        x,y = np.random.choice(2, 2)
        return [x,y]

def make_smaller_(l):
    '''-2 to -1, 2 to 1'''
    for i_nr, i in enumerate(l):
        if i == -2:
            l[i_nr] = -1
        elif i == 2:
            l[i_nr] = 1
    return l
  
def normalize_position_if_outside_world(position, max_border):
    '''limit position to world border'''
    if position < 0:
        position = 0
    elif position > max_border:
        position = max_border
    return position


In [ ]:
calculate_position(result, indiv, x, y, world_size_x, world_size_y)